# 〇、 依赖项

In [38]:
import cv2
import os
import numpy as np
from pandas import Series,DataFrame
import pandas as pd

# 一、特征
## 1、hash算法——与像素点的位置相关
### 1） 均值hash算法
将图片转换为灰度图，将图片尺寸调整为8\*8，计算平均值，根据均值获得图像指纹，即像素值大于均值的设为True，反之为False。

In [3]:
def aHash(img):
	img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
	img=cv2.resize(img,(8,8), interpolation=cv2.INTER_AREA) 
	means=img.sum()/64
	img=np.where(img>means,True,False)
	return img 

### 2）感知hash算法
将图片转换为灰度图，将图片尺寸调整为32\*32，做DCT变换，获取结果左上角尺寸为8\*8的部分，根据均值获得图像指纹，即像素值大于均值的设为True，反之为False。

In [4]:
def pHash(img):
	img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
	img=cv2.resize(img,(32,32), interpolation=cv2.INTER_AREA) 
	img= cv2.dct(np.float32(img))
	img = img[0:8,0:8] 
	means=img.sum()/64
	img=np.where(img>means,True,False)
	return img 	

### 3）差值Hash算法
比感知hash快，准确性理论上高于均值hash
将图片转换为灰度图，将图片尺寸调整为9\*8，除了最后一列像素，其他每个像素只要值比右边的像素小，则设为True，反之则为False

In [7]:
def dHash(img):
	img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
	img=cv2.resize(img,(8,9), interpolation=cv2.INTER_AREA) 
	hash=np.zeros((8,8),dtype=bool)
	for i in range(8):
		for j in range(8):
			if img[i][j]<img[i+1][j]:
				hash[i][j]=True
			else:
				hash[i][j]=False
	return hash

## 2、颜色矩——统计结果，与色彩有关
首先，将原图转换为YUV色彩空间，因为这更符合人眼的感知。
分别计算各个通道的一阶矩，二阶矩，三阶矩，组成一个九维向量

In [8]:
def color_moment(img):
	[width,height,channel]=img.shape
	#YUV色彩空间，各个通道像素值统计
	y=Y_vector(img)
	u=U_vector(img)
	v=V_vector(img)
	#一阶矩，均值
	ay=y.sum()/width/height
	au=u.sum()/width/height
	av=v.sum()/width/height
	by=0
	bu=0
	bv=0
	cy=0
	cu=0
	cv=0
	#计算二阶矩和三阶矩
	for i in range(256):
		by=by+(i-ay)**2/width/height*y[i]
		bu=bu+(i-au)**2/width/height*u[i]
		bv=bv+(i-av)**2/width/height*v[i]
		cy=cy+(i-ay)**3/width/height*y[i]
		cu=cu+(i-au)**3/width/height*u[i]
		cv=cv+(i-av)**3/width/height*v[i]
	by=by**0.5
	bu=bu**0.5
	bv=bv**0.5
	cy=cy**(1/3)
	cu=cu**(1/3)
	cv=cv**(1/3)
	#将YUV三通道的三种矩组成一个九维向量
	vector=np.zeros(9)
	vector[0]=ay
	vector[1]=by
	vector[2]=cy
	vector[3]=au
	vector[4]=bu
	vector[5]=cu
	vector[6]=av
	vector[7]=bv
	vector[8]=cv
	return vector

### Y通道像素值统计	

In [16]:
def Y_vector(img):
	img=cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
	img=img[:,:,0]
	[width,height]=img.shape
	v=np.zeros(256)
	for i in range(width):
		for j in range(height):
			p=img[i,j]
			v[p]=v[p]+1/width/height
	return v

### U通道像素值统计

In [17]:
def U_vector(img):
	img=cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
	img=img[:,:,1]
	[width,height]=img.shape
	v=np.zeros(256)
	for i in range(width):
		for j in range(height):
			p=img[i,j]
			v[p]=v[p]+1/width/height
	return v

### V通道像素值统计

In [11]:
def V_vector(img):
	img=cv2.cvtColor(img,cv2.COLOR_BGR2YUV)
	img=img[:,:,2]
	[width,height]=img.shape
	v=np.zeros(256)
	for i in range(width):
		for j in range(height):
			p=img[i,j]
			v[p]=v[p]+1/width/height
	return v

## 3、灰度直方图——统计结果，与色彩无关
获得一个256维向量

In [12]:
def grey_vector(img):
	img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
	[width,height]=img.shape
	v=np.zeros(256)
	for i in range(width):
		for j in range(height):
			p=img[i,j]
			v[p]=v[p]+1/width/height
	return v

# 二、相似度衡量
## 1、向量的余弦相似度

In [13]:
def cos_similarity(v1,v2):
	if v1.shape!=v2.shape:
		return False
	similarity=np.dot(v1,v2)/np.linalg.norm(v1)/np.linalg.norm(v2)
	return similarity

## 2、向量的闵科夫斯基相似度
当p值为1时，结果为曼哈顿距离；为2时，结果为欧式距离

In [18]:
def Minkowski_similarity(v1,v2,p):
	if v1.shape!=v2.shape:
		return False
	similarity=(abs(v1-v2)**p).sum()**(1/p)
	return similarity

## 3、汉明距离
仅适用于hash算法
对两个向量的每位取异或，统计结果的True的数量，用来表示两个二值向量的相似度

In [15]:
def hamming(v1,v2):
	if v1.shape!=v2.shape:
		return False
	v3=v1^v2
	d=0
	for index in v3:
		for i in index:
			if i:
				d=d+1
	return d

# 三、运行

In [36]:
img=cv2.imread("./img/1.jpg",cv2.IMREAD_COLOR) 
u1=aHash(img)
u2=pHash(img)
u3=dHash(img)
u4=color_moment(img)
u5=grey_vector(img)
root = ".\\img"
result={"file_name":[],"ahash":[],"phash":[],"dhash":[],"color_cos":[],"color_Euclid":[],"color_man":[],"grey_cos":[],"grey_Euclid":[],"grey_man":[]}
for dirpath, dirnames, filenames in os.walk(root):
	for filepath in filenames:
		name=os.path.join(dirpath, filepath)
		img2=cv2.imread(name,cv2.IMREAD_COLOR)
		v1=aHash(img2)
		v2=pHash(img2)
		v3=dHash(img2)
		v4=color_moment(img2)
		v5=grey_vector(img2)
		result["file_name"].append(name)
		result["ahash"].append(hamming(u1,v1))
		result["phash"].append(hamming(u2,v2))
		result["dhash"].append(hamming(u3,v3))
		result["color_cos"].append(cos_similarity(u4,v4))
		result["grey_cos"].append(cos_similarity(u5,v5))
		result["color_Euclid"].append(Minkowski_similarity(u4,v4,2))
		result["grey_Euclid"].append(Minkowski_similarity(u5,v5,2))
		result["color_man"].append(Minkowski_similarity(u4,v4,1))
		result["grey_man"].append(Minkowski_similarity(u5,v5,1))
df = DataFrame(result)

In [37]:
df

,ahash,color_Euclid,color_cos,color_man,dhash,file_name,grey_Euclid,grey_cos,grey_man,phash
0,0,0.000000,1.000000,0.000000,0,.\img\1.jpg,0.000000,1.000000,0.000000,0
1,17,1.300273,0.991472,2.060728,24,.\img\10.jpg,0.088253,0.985189,0.536835,10
2,8,1.125511,0.998970,2.059242,13,.\img\2.jpg,0.170348,0.994870,0.823096,10
3,15,0.805074,0.995285,1.195058,23,.\img\3.jpg,0.414277,0.675938,1.358062,13
4,8,1.213637,0.993331,2.198481,14,.\img\4.jpg,0.533130,0.067028,1.054340,11
5,15,0.730360,0.998133,1.270174,30,.\img\5.jpg,0.493159,0.044561,1.591107,11
6,14,1.712708,0.998500,3.321663,25,.\img\6.jpg,0.057893,0.998375,0.311428,13
7,13,0.695796,0.997877,1.343188,17,.\img\7.jpg,0.481508,0.134637,0.980957,13
8,44,0.915988,0.987954,1.672933,47,.\img\8.jpg,0.486540,0.067507,1.236686,21
9,44,2.125764,0.851835,2.828300,31,.\img\9.jpg,0.548195,0.005336,1.703660,32


# 四、结果分析
注：图片`1.jpg`就是被比较的图片
## 1、各个结果中最相似图片分别为：
|均值hash|感知hash|差值hash|颜色矩-余弦相似度|颜色矩-欧式距离|颜色矩-曼哈顿距离|灰度-余弦相似度|灰度-欧式距离|灰度-曼哈顿距离|
|-|-|-|-|-|-|-|-|-|
|2、4|2、10|2|2|7|3|6|6|6|

## 2、 分析
- 结合各个结果，2号图片和6号图片均与1号图片较为相似。

1号图片：![img](./img/1.jpg)2号图片：![img](./img/2.jpg)6号图片：![img](./img/6.jpg)

- hash算法都指向了2号图片，因为这两种算法考虑了空间关系
- 灰度直方图的方法都指向了6号图片，是因为这张图片的灰度统计量和1号图片相似，但是对于人眼来说，这种统计评价方法并不直观
- 颜色矩的方法，在采用不同相似度比较方法时，结果迥异，甚至将3号图片（颜色矩-曼哈顿距离）选中（虽然3号图片的质感和1号图片十分相似）![img](./img/3.jpg)
- 感知hash出现了一个很离奇的结果，10号图片![img](./img/10.jpg)
可能是因为10号图片的低频信息和1号图片很接近，但依然对于人眼来说不直观

### 3、总结
本次实验观察了五种图像信息，用了四种向量（矩阵）相似度的比较方法。
最后总体结果还是比较理想的。但还是缺少一个策略，将这些方法综合起来应用，因为缺少一个图片比较的目标。